In [0]:
from pyspark.sql import functions as F

In [0]:
spark.conf.set("spark.sql.session.timeZone", "America/Sao_Paulo")

In [0]:
origem_path = 'fomacao_microsoft_power_bi_profisional.bronze.pessoas'

In [0]:
df_silver = (
    spark.read.table(origem_path)
    .withColumn('_ingest_ts_utc', F.current_timestamp())
    .withColumn("_ingest_date", F.to_date(F.col("_ingest_ts_utc")))
    .withColumn('NomeCompleto', F.concat_ws(' ', F.col('PrimeiroNome'), F.col('UltimoNome')))
    .withColumn('Genero',
                F.when(F.col('Genero') == 'Male', 'Masculino')
                 .when(F.col('Genero') == 'Female', 'Feminino')
                 .otherwise(F.col('Genero'))
                )
    .withColumn('login', F.split(F.col('email'), '@').getItem(0))
    .withColumn('dominio', F.upper(F.split(F.col('email'), '@').getItem(1)))
    .withColumn('AnoNascimento', F.year(F.col('Nascimento')))
    .drop(
        '_source_path', 
        '_source_file', 
        'PrimeiroNome', 
        'UltimoNome'
        )
    .select(
        'Codigo', 
        'email',
        'login', 
        'dominio',
        'NomeCompleto', 
        'Genero', 
        'Avatar', 
        'Pais', 
        'Nascimento', 
        'AnoNascimento',
        '_ingest_ts_utc', 
        '_ingest_date'
        )
    )

In [0]:
tbl = 'pessoas'
destino_path = f'fomacao_microsoft_power_bi_profisional.silver.{tbl}'

df_silver = (df_silver.write
             .format('delta')
             .mode('overwrite')
             .option('overwriteSchema', 'true')
             .partitionBy('_ingest_date')
             .saveAsTable(destino_path))

print(f'Contagem de registros: {spark.table(destino_path).count()}')